In [ ]:
import kagglehub
from seaborn import heatmap

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd

In [ ]:
books = pd.read_csv(f"{path}/books.csv")

In [ ]:
books

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
ax = plt.axes()
sns.heatmap(books.isna().transpose(),cbar=False,ax=ax)
plt.xlabel("Columns")
plt.ylabel("Missing values")
plt.show()

In [ ]:
import numpy as np

books["missing_description"] = np.where(books["description"].isna(), 1, 0)
books["age_of_book"] = 2025 - books["published_year"]

In [ ]:
columns_of_interest = ["num_pages","age_of_book","missing_description","average_rating"]
correlation_matrix = books[columns_of_interest].corr(method="spearman")

sns.set_theme(style="white")
plt.figure(figsize=(8,6))
heatmap = sns.heatmap(correlation_matrix, annot=True,fmt=".2f",cmap="coolwarm",cbar_kws={"label":"Spearman Correlation"})
heatmap.set_title("Correlation Heatmap")
plt.show()

In [ ]:
final_books = books[
    ~(books['description'].isna()) &
    ~(books['num_pages'].isna()) &
    ~(books['average_rating'].isna()) &
    ~(books['published_year'].isna())
]

In [ ]:
final_books

In [ ]:
final_books["categories"].value_counts().reset_index().sort_values("count", ascending=False)

In [ ]:
final_books

In [ ]:
final_books.loc[:,"words_in_description"] = final_books["description"].str.split().str.len()

In [ ]:
final_books

In [ ]:
final_books.loc[final_books["words_in_description"].between(1,4),"description"]

In [ ]:
final_books_25_words = final_books[final_books["words_in_description"] >= 25]

In [ ]:
final_books_25_words

In [ ]:
final_books_25_words.loc[:,"title_and_subtitle"] = (
    np.where(final_books_25_words["subtitle"].isna(),final_books_25_words["title"],
             final_books_25_words[["title","subtitle"]].astype(str).agg(" ".join,axis=1))
)

In [ ]:
final_books_25_words

In [ ]:
final_books_25_words.loc[:,"tagged_description"] = final_books_25_words[["isbn13","description"]].astype(str).agg(" ".join,axis=1)

In [ ]:
final_books_25_words

In [ ]:
(
    final_books_25_words
    .drop(["subtitle","missing_description","age_of_book","words_in_description"],axis=1)
    .to_csv("books_cleaned.csv",index=False)
)